In [30]:
%matplotlib notebook
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import math
from collections import Counter
from sklearn.preprocessing import minmax_scale
from scipy.spatial.distance import pdist, squareform
from time import time
df = pd.read_csv('WDI_csv/WDIData.csv',index_col='Country Name')
df = df.iloc[:, :-1]


In [43]:
# df.fillna(0, inplace=True)

def distance_matrix(X, f='correlation'):
    return squareform(pdist(X, metric=f))

def get_interpoled_data(datas):
    # TODO: make it work
    for i in range(datas.shape[0]):
        original_row = datas.iloc[i, :]
        curated_row = original_row.dropna()

        if len(curated_row) < 2:
            raise ValueError("Need at least 2 not nane values!")


        indexes = original_row.index.values
        indexes = indexes.astype(np.float)

        x = curated_row.index.values
        x = x.astype(np.float)
        y = curated_row.values
        f = interp1d(x, y, fill_value='extrapolate')

        for n in range(len(indexes)):
            if math.isnan(datas.iat[i,n]):
                calcul = f(indexes[n])
                datas.iat[i,n] = calcul if calcul > 0 else 0
    return datas
    
def get_data_by_name(name):
    data = df[df['Indicator Name'] == name]
    data = data.iloc[:, 3:]
    return ((get_interpoled_data(data).values), data.index)
#     return(minmax_scale(data.values, axis=0), data.index)

    

In [4]:
def get_most_common_n(n):
    a = df[df['1960'].notnull()]['Indicator Name']
    test = Counter()
    for b, c in zip(a, a.index):
        test[b] += 1
    return test.most_common(n)

In [5]:
set(df.loc['Afghanistan']['Indicator Name'])

{'PPG, bilateral concessional (TDS, current US$)',
 'Population ages 40-44, male (% of male population)',
 'General government final consumption expenditure (current LCU)',
 'GNI (constant 2010 US$)',
 'Public private partnerships investment in ICT (current US$)',
 'Access to electricity, urban (% of urban population)',
 'Average interest on new external debt commitments, private (%)',
 'Smoking prevalence, males (% of adults)',
 'Land under cereal production (hectares)',
 'People using at least basic sanitation services, urban  (% of urban population)',
 'Population ages 75-79, male (% of male population)',
 'Population ages 75-79, female (% of female population)',
 'International tourism, number of departures',
 'Final consumption expenditure (current US$)',
 'Pupil-teacher ratio, tertiary',
 'Employers, female (% of female employment) (modeled ILO estimate)',
 'Population ages 45-49, female (% of female population)',
 'Net primary income (BoP, current US$)',
 'Customs and other impo

In [18]:
T = get_most_common_n(333)
test_pool = []
for a in range(0, 120, 4):
    if not ('population' in T[a][0].lower() or 'survival'in T[a][0].lower()):
        test_pool.append(T[a][0])
test_pool
    

['Mortality rate, adult, male (per 1,000 male adults)',
 'Life expectancy at birth, total (years)',
 'CO2 emissions from liquid fuel consumption (% of total)',
 'CO2 emissions from liquid fuel consumption (kt)',
 'CO2 emissions from solid fuel consumption (kt)',
 'Net official development assistance and official aid received (current US$)',
 'Net official development assistance received (constant 2015 US$)',
 'Merchandise exports by the reporting economy, residual (% of total merchandise exports)',
 'Merchandise imports from high-income economies (% of total merchandise imports)',
 'Net bilateral aid flows from DAC donors, Total (current US$)',
 'Merchandise imports from low- and middle-income economies in South Asia (% of total merchandise imports)']

In [44]:
in_list = []
for name in test_pool[2:]:
    try:
        in_list.append(get_data_by_name(name))
    except ValueError:
        print(name)
        continue
in_list

CO2 emissions from liquid fuel consumption (% of total)
CO2 emissions from liquid fuel consumption (kt)
CO2 emissions from solid fuel consumption (kt)
Net official development assistance and official aid received (current US$)
Net official development assistance received (constant 2015 US$)
Merchandise exports by the reporting economy, residual (% of total merchandise exports)
Merchandise imports from high-income economies (% of total merchandise imports)
Net bilateral aid flows from DAC donors, Total (current US$)
Merchandise imports from low- and middle-income economies in South Asia (% of total merchandise imports)


[]

In [ ]:
get_interpoled_data(datas)

In [ ]:
datas = datas.iloc[:, 3:]
datas = datas.dropna(axis='columns', how='all')
datas = datas.dropna(axis='index', how='any')
X = datas.values

In [ ]:
datas.index

In [ ]:
from dtaidistance import dtw
from sklearn.preprocessing import normalize
X = normalize(X, norm='max')
ds = dtw.distance_matrix_fast(X)
ds[ds == np.inf] = 0
ds

In [ ]:
from dtaidistance import clustering
# Custom Hierarchical clustering
model1 = clustering.Hierarchical(dtw.distance_matrix_fast, {})
# Augment Hierarchical object to keep track of the full tree
model2 = clustering.HierarchicalTree(model1)
# SciPy linkage clustering
model3 = clustering.LinkageTree(dtw.distance_matrix_fast, {})
cluster_idx1 = model3.fit(datas.values[:20,:])

# cluster_idx2 = model2.fit(datas.values)

In [ ]:

import matplotlib.pyplot as plt
from sklearn import manifold
from time import time

fig, ax = plt.subplots(figsize=(8,6))
t0 = time()
mds = manifold.TSNE(2, init='pca', random_state=0)
Y = mds.fit_transform(ds)
t1 = time()
color = [i for i in range(len(Y))]
ax.scatter(Y[:, 0], Y[:, 1], c=color, cmap=plt.cm.Spectral)
plt.title("TSNE (%.2g sec)" % (t1 - t0))

for i, txt in enumerate(datas.index):
    ax.annotate(txt, (Y[i, 0], Y[i, 1]))
plt.show()

In [ ]:
import pyximport; pyximport.install()
from frechet import frechet

In [ ]:
P = datas.values[0].copy(order='C')
Q = datas.values[1].copy(order='C')
frechet(P, Q)

In [ ]:
Y